In [ ]:
%%writefile submission.py

import math
import random

Qt = [[[0,0,0],[0,0,0],[0,0,0]],
      [[0,0,0],[0,0,0],[0,0,0]],
      [[0,0,0],[0,0,0],[0,0,0]]]

Nt = [[[0,0,0],[0,0,0],[0,0,0]],
      [[0,0,0],[0,0,0],[0,0,0]],
      [[0,0,0],[0,0,0],[0,0,0]]]

my_hand = 0
my_hand_2 = 0
opp_hand_2 = 0

def agent(obs, conf):
    global Qt, Nt, my_hand, my_hand_2, opp_hand_2
    """
    Select an action based on the Formula:
    At = argmax(a, [Qt(a) + c * sqrt(ln(t)/Nt(a))])
    Qt(a) -> Expected Reward for the Action a
    c -> Confidence level (similar to epsilon)
    Nt(a) -> Number of times an action has been used at time step t
    """
    if obs['step'] < 1:
        my_hand_2 = random.choice([0,1,2])
        return my_hand_2
    elif obs['step'] < 2:
        my_hand = random.choice([0,1,2])
        opp_hand_2 = obs['lastOpponentAction']
        #Update Nt
        Nt[my_hand_2][opp_hand_2][my_hand] += 1
        return my_hand
    else:
        opp_hand = obs['lastOpponentAction']

        #Update Qt
        for count, action in enumerate(Qt[my_hand_2][opp_hand_2]):
            reward = 0
            if (opp_hand + 1) % 3 == count:
                reward = 1
            elif opp_hand == count:
                reward = 0    
            else:
                reward =  -1

            Qt[my_hand_2][opp_hand_2][count] += reward
        #Get Next Action
        max_return = -1000
        next_action = 0
        for action in range(len(Qt[my_hand][opp_hand])):
            #To Avoid Divide by Zero, return all actions that have not been taken
            #To ensure all actions are taken atleast once
            if Nt[my_hand][opp_hand][action] == 0:
                Nt[my_hand][opp_hand][action] += 1
                my_hand_2 = my_hand
                opp_hand_2 = opp_hand
                my_hand = action
                return action
            else:
                c = 1.0
                t = obs['step']
                this_return = Qt[my_hand][opp_hand][action] + math.sqrt(math.log(t) / Nt[my_hand][opp_hand][action])

                if this_return > max_return:
                    max_return = this_return
                    next_action = action
        
        Nt[my_hand][opp_hand][next_action] += 1
        my_hand_2 = my_hand
        opp_hand_2 = opp_hand
        my_hand = next_action
        return next_action
        